In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from functools import partial
from sklearn.model_selection import train_test_split

In [2]:
products = []
customers = []
for i in range(3):
    product = TimeSeriesGenerator(size=1460, amountOfVariables=7)
    customer = TimeSeriesGenerator(size=1460, amountOfVariables=3)
    products.append(product)
    customers.append(customer)


In [3]:
thesis = ExperimentTracker(products, customers, scenarios())

In [4]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), make_column_selector(dtype_include=object))], remainder='passthrough')
xgb_pipe = Pipeline([
    ('onehot', onehot_cols),
('xgb', XGBRegressor())
])

In [5]:
thesis.runExperiment(MLalgo=xgb_pipe)   

Finished experiment! Elapsed time: 2.6020889282226562, total Elapsed time: 2.6020889282226562, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 2.6730058193206787, total Elapsed time: 5.275094747543335, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 2.3477776050567627, total Elapsed time: 7.622872352600098, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 2.457810401916504, total Elapsed time: 10.080682754516602, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished experiment! Elapsed time: 2.483273983001709, total Elapsed time: 12.56395673751831, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observ

In [10]:
results = thesis.resultsToDF()

In [11]:
results.groupby(["Drift type", "Drift time", "Drift magnitude"]).mean()

MSE      MAPE  \
Drift type        Drift time     Drift magnitude                           
Incremental Drift Fully observed Large               18.388101  0.014866   
                                 Small               12.160362  0.014047   
                  Half observed  Large              157.497164  0.014403   
                                 Small               17.545903  0.013727   
                  Unobserved     Large              155.772685  0.014405   
                                 Small               17.272966  0.013721   
No Drift          No Drift       No Drift            10.371126  0.013355   
Sudden Drift      Fully observed Large               23.142253  0.009746   
                                 Small               11.221367  0.011527   
                  Half observed  Large               75.587452  0.016624   
                                 Small               28.729461  0.016374   
                  Unobserved     Large            54099.329567  0.394642   
                                 Small             2694.956215  0.152789   

                                                      SMAPE  
Drift type        Drift time     Drift magnitude             
Incremental Drift Fully observed Large             1.493333  
                                 Small             1.410667  
                  Half observed  Large             1.482000  
                                 Small             1.380000  
                  Unobserved     Large             1.482667  
                                 Small             1.380667  
No Drift          No Drift       No Drift          1.338000  
Sudden Drift      Fully observed Large             0.978667  
                                 Small             1.154000  
                  Half observed  Large             1.644667  
                                 Small             1.635333  
                  Unobserved     Large            53.695333  
                                 Small            17.248667

In [8]:
results.sort_values("SMAPE", ascending=False)

,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,MAPE,SMAPE
159,"[0, 3, 6]",Sudden Drift,Large,Important,Unobserved,116511.379437,0.608384,88.45
11,None,Sudden Drift,Large,Important,Unobserved,116430.311850,0.608254,88.43
48,0,Sudden Drift,Large,Important,Unobserved,116431.321326,0.608186,88.40
122,6,Sudden Drift,Large,Important,Unobserved,116117.535113,0.607283,88.23
85,3,Sudden Drift,Large,Important,Unobserved,115878.645339,0.606848,88.12
...,...,...,...,...,...,...,...,...
121,6,Sudden Drift,Large,Important,Half observed,11.715375,0.005231,0.52
10,None,Sudden Drift,Large,Important,Half observed,11.389325,0.005138,0.51
120,6,Sudden Drift,Large,Important,Fully observed,10.473129,0.004997,0.50
83,3,Sudden Drift,Large,Important,Fully observed,10.449119,0.004976,0.50


In [12]:
results.to_csv("first_resultsWorking.csv")